# Collect WUQI Data

A web crawler is used to collect weather information for WUQI.

Data from : http://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station=467770&stname=%25E6%25A2%25A7%25E6%25A3%25B2&datepicker=2016-01-01

In [1]:
#import module
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import numpy as np
import time

In [2]:
#function to collect weather information
def append_new_data(ori_data, url, append_date):
    time.sleep(0.5)
    url_date = url + str(append_date)[:11]
    end_date = pd.to_datetime([1], unit='D', origin=pd.Timestamp(append_date))
    time_hour = pd.date_range(start=append_date, end=end_date[0], freq='H')[1:]
    html = urlopen(url_date).read()
    soup = BeautifulSoup(html, "lxml")
    # get columns text
    c_tag = soup.find_all(class_='second_tr')
    c_text = [i.get_text().split('\n') for i in c_tag][0]
    c_text = [i.replace('\t', '') for i in c_text if i != '']
    # get data value
    v_tag = soup.find_all('td')
    v_value = [i.get_text().replace('\xa0', '') for i in v_tag]
    data_value = np.array(v_value[10:]).reshape(24, -1)
    append_df = pd.DataFrame(data_value, columns=c_text, index=time_hour)
    new_df = pd.concat([ori_data, append_df])
    return new_df

In [4]:
#run function to collect data
url = "http://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station=467770&stname=%25E6%25A2%25A7%25E6%25A3%25B2&datepicker="
start_date = '2016-01-01'
final_date = '2017-12-31'

my_df = pd.DataFrame()
time_delta = pd.date_range(start=start_date, end=final_date, freq='D')
for date_i in time_delta:
    my_df = append_new_data(my_df, url, date_i)
#     print(date_i, 'is finished')
# If you run this block, wait 5 minutes.


In [5]:
# save data
Location = 'WUQI '
my_df.to_csv('data\\'+Location+'.csv', encoding='utf-8-sig')
print(my_df)

                    觀測時間(hour) 測站氣壓(hPa) 海平面氣壓(hPa) 氣溫(℃) 露點溫度(℃) 相對溼度(%)  \
2016-01-01 01:00:00         01    1021.7     1024.9  15.1    10.8      75   
2016-01-01 02:00:00         02    1021.5     1024.7  15.0    10.8      76   
2016-01-01 03:00:00         03    1021.0     1024.2  14.9    10.7      76   
2016-01-01 04:00:00         04    1021.1     1024.3  14.9    10.9      77   
2016-01-01 05:00:00         05    1021.0     1024.2  14.9    11.1      78   
...                        ...       ...        ...   ...     ...     ...   
2017-12-31 20:00:00         20    1017.3     1020.5  15.0     9.6      70   
2017-12-31 21:00:00         21    1017.4     1020.6  15.1     9.4      69   
2017-12-31 22:00:00         22    1017.5     1020.7  15.1     9.2      68   
2017-12-31 23:00:00         23    1017.4     1020.6  15.3     9.3      67   
2018-01-01 00:00:00         24    1017.3     1020.5  15.2     9.4      68   

                    風速(m/s) 風向(360degree) 最大陣風(m/s) 最大陣風風向(360degree) 降水量(m